In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from datetime import timedelta

In [42]:
'''Create empty Dataframe'''
my_index = pd.MultiIndex(levels=[[],[],[]],
                         codes=[[],[],[]],
                             names=[u'STime', u'Point', u'Index'])
my_columns = [u'Year', u'Month',u'Day',u'Hour',u'Minute',u'Second',u'x',u'y',u'theta',u'Record',u'U (m s^-1)',u'V (m s^-1)',u'W (m s^-1)',u'TSonic (K)',u'RH (%)',u'T_HMP60 (C)']
data = pd.DataFrame(index=my_index, columns=my_columns)

'''IMPORT DATA'''
files = ['16-00','16-30','17-00','17-30','18-00','18-30','19-00','19-30',
            '20-00','20-30','21-00','21-30','22-00','22-30','23-00','23-30',
            '00-00','00-30','01-00','01-30','02-00','02-30','03-00','03-30',
            '04-00','04-30','05-00','05-30','06-00','06-30','07-00','07-30',
            '08-00','08-30','09-00','09-30','10-00','10-30','11-00','11-30',
            '12-00','12-30','13-00','13-30','14-00','14-30','15-00','15-30']

for i in files:
    temppd = pd.read_csv('./Data/Binned Data/'+i+'.csv',index_col=[0,1])
    
    #append dataframe into multiframe
    data = data.append(temppd.assign(k=i).set_index('k',append=True).swaplevel(0,2).swaplevel(1,2))
    
#data = data.sort_index() #sort cuz faster

In [50]:
points=['p1','p2','p3','p4','p5','p6','p7','p8']

my_index = pd.MultiIndex(levels=[[],[]],
                         codes=[[],[]],
                             names=[u'STime', u'Point'])
my_columns = [u'Uavg',u'Vavg',u'Wavg','Savg',u'TSonicavg',u'RHavg',u'T_HMP60avg',u'Uvar',u'Vvar',u'Wvar',u'k',
             u'UVvar',u'UWvar',u'VWvar',u'UTvar',u'VTvar',u'WTvar']

p_data = pd.DataFrame(index=my_index, columns=my_columns)

for dtime in tqdm(files):
    for point in points:
        p_data.loc[(dtime,point),'Uavg'] = np.mean(data.loc[(dtime,point),'U (m s^-1)'])
        p_data.loc[(dtime,point),'Vavg'] = np.mean(data.loc[(dtime,point),'V (m s^-1)'])
        p_data.loc[(dtime,point),'Wavg'] = np.mean(data.loc[(dtime,point),'W (m s^-1)'])
        p_data.loc[(dtime,point),'TSonicavg'] = np.mean(data.loc[(dtime,point),'TSonic (K)'])-273.15 #shift to Celcius
        p_data.loc[(dtime,point),'RHavg'] = np.mean(data.loc[(dtime,point),'RH (%)'])
        p_data.loc[(dtime,point),'T_HMP60avg'] = np.mean(data.loc[(dtime,point),'T_HMP60 (C)'])
        p_data.loc[(dtime,point),'Savg'] = np.mean(np.sqrt(data.loc[(dtime,point),'U (m s^-1)']**2+
                                                           data.loc[(dtime,point),'V (m s^-1)']**2))
        
        
        x = [j for j in range(0, len(data.loc[(dtime,point),'U (m s^-1)']))]
        U = data.loc[(dtime,point),'U (m s^-1)']
        Umodel = np.polyfit(x,U,1)
        Utrend = np.polyval(Umodel,x)
        Udetrended = U - Utrend
        V = data.loc[(dtime,point),'V (m s^-1)']
        Vmodel = np.polyfit(x,V,1)
        Vtrend = np.polyval(Vmodel,x)
        Vdetrended = V - Vtrend
        W = data.loc[(dtime,point),'W (m s^-1)']
        Wmodel = np.polyfit(x,W,1)
        Wtrend = np.polyval(Wmodel,x)
        Wdetrended = W - Wtrend
        TSonic =  data.loc[(dtime,point),'TSonic (K)']
        TSonicmodel = np.polyfit(x, TSonic,1)
        TSonictrend = np.polyval( TSonicmodel,x)
        TSonicdetrended =  TSonic -  TSonictrend
        
        UVCovMatrix = np.cov(Udetrended, Vdetrended)
        UWCovMatrix = np.cov(Udetrended, Wdetrended)
        VWCovMatrix = np.cov(Vdetrended, Wdetrended)
        UTSonicCovMatrix = np.cov(Udetrended, TSonicdetrended)
        VTSonicCovMatrix = np.cov(Vdetrended, TSonicdetrended)
        WTSonicCovMatrix = np.cov(Wdetrended, TSonicdetrended)

        p_data.loc[(dtime,point),'Uvar'] = UVCovMatrix[0,0]
        p_data.loc[(dtime,point),'Vvar'] = UVCovMatrix[1,1]
        p_data.loc[(dtime,point),'Wvar'] = UWCovMatrix[1,1]
        p_data.loc[(dtime,point),'TSonicvar'] = UTSonicCovMatrix[1,1]
        p_data.loc[(dtime,point),'k'] = 1/2*(p_data.loc[(dtime,point),'Uvar']+p_data.loc[(dtime,point),'Vvar']+p_data.loc[(dtime,point),'Wvar'])

        p_data.loc[(dtime,point),'UVvar'] = UVCovMatrix[0,1]
        p_data.loc[(dtime,point),'UWvar'] = UWCovMatrix[0,1]
        p_data.loc[(dtime,point),'VWvar'] = VWCovMatrix[0,1]
        p_data.loc[(dtime,point),'UTvar'] = UTSonicCovMatrix[0,1]
        p_data.loc[(dtime,point),'VTvar'] = VTSonicCovMatrix[0,1]
        p_data.loc[(dtime,point),'WTvar'] = WTSonicCovMatrix[0,1]

    
print(p_data)
    
    
    
'''Vavg = np.mean(V4Hz[i*AverageSample:(i+1)*AverageSample])
Wavg = np.mean(W4Hz[i*AverageSample:(i+1)*AverageSample])
TSonicavg = np.mean(TSonic4Hz[i*AverageSample:(i+1)*AverageSample])
Savg = np.mean(np.sqrt(U4Hz[i*AverageSample:(i+1)*AverageSample] ** 2 + \
                            V4Hz[i*AverageSample:(i+1)*AverageSample] ** 2))'''

100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:28<00:00,  1.69it/s]

                  Uavg        Vavg         Wavg       Savg TSonicavg    RHavg  \
STime Point                                                                     
16-00 p1    -0.0150259  -0.0120259    0.0336278  0.0669413   23.1583  33.1996   
      p2    -0.0067214  -0.0133438 -0.000298278  0.0247344    23.003  33.2299   
      p3    -0.0369643 -0.00323063   0.00647847   0.043681   23.0642  33.0189   
      p4    -0.0269671  -0.0226333     0.103981  0.0591891   23.6655  33.0794   
      p5    -0.0421473 -0.00833579    0.0222627  0.0598653   23.0911  32.6345   
...                ...         ...          ...        ...       ...      ...   
15-30 p4     -0.014525  -0.0176965    0.0916329  0.0589947    23.465   33.481   
      p5    -0.0481912  0.00336767   0.00900432  0.0517842   23.0746  33.0722   
      p6    -0.0319208 -0.00584292    0.0215807  0.0399221   23.0643  33.0788   
      p7    -0.0338166  -0.0661242   -0.0220488  0.0918856   22.9612    33.01   
      p8    -0.0315461   -0.

'Vavg = np.mean(V4Hz[i*AverageSample:(i+1)*AverageSample])\nWavg = np.mean(W4Hz[i*AverageSample:(i+1)*AverageSample])\nTSonicavg = np.mean(TSonic4Hz[i*AverageSample:(i+1)*AverageSample])\nSavg = np.mean(np.sqrt(U4Hz[i*AverageSample:(i+1)*AverageSample] ** 2 +                             V4Hz[i*AverageSample:(i+1)*AverageSample] ** 2))'

In [32]:
data.loc[(dtime,point),'U (m s^-1)']

C:\Users\jdiro\anaconda3\lib\site-packages\pandas\core\indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


Index
13339   -0.248
13340   -0.247
13341   -0.096
13342   -0.096
13343   -0.096
         ...  
14951    0.012
14952    0.012
14953    0.012
14954    0.012
14955    0.012
Name: U (m s^-1), Length: 1617, dtype: float64

In [48]:
len(data.loc[(dtime,point),'U (m s^-1)'])

1623